In [1]:
import pandas as pd
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# References:
- 


In [3]:
root = './RASA/data/'
processed = './Data/processing/Processed_Airbnb/'
raw = './Data/raw/'

def create_embeddings(text_corpus_listing, name):
    processed_text_corpus = preprocess_documents(text_corpus_listing)
    tagged_text_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(processed_text_corpus)]
    text_corpus_model = Doc2Vec(tagged_text_corpus, dm=0, vector_size=200, window=2, min_count=1, epochs=100, hs=1)

    text_corpus_model.save(root+'embeddings/'+ name +'_embeddings')

    return text_corpus_model

### User Review Embeddings

In [4]:
reviews = pd.read_csv(processed+'ratings_filter.csv', sep=',')


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
reviews = reviews.dropna()

In [6]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264360 entries, 0 to 272484
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     264360 non-null  object 
 1   listing_id     264360 non-null  float64
 2   id             264360 non-null  float64
 3   date           264360 non-null  object 
 4   reviewer_id    264360 non-null  float64
 5   reviewer_name  264360 non-null  object 
 6   comments       264360 non-null  object 
 7   rating         264360 non-null  float64
dtypes: float64(4), object(4)
memory usage: 18.2+ MB


In [7]:
comment_corpus_model = create_embeddings(reviews['comments'].values,'review')

In [8]:
new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher")
test_doc_vector = comment_corpus_model.infer_vector(new_doc)
sims = comment_corpus_model.docvecs.most_similar(positive = [test_doc_vector])
for s in sims:
    print(f"{(s[1])} | {reviews['listing_id'].iloc[s[0]]}")

<ipython-input-8-25588e9477fc>:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = comment_corpus_model.docvecs.most_similar(positive = [test_doc_vector])


0.6857652068138123 | 23626417.0
0.6626643538475037 | 35632344.0
0.6593735814094543 | 31945057.0
0.6566953063011169 | 37927536.0
0.6455527544021606 | 35855352.0
0.6263290643692017 | 30169743.0
0.6258693337440491 | 27921739.0
0.6225862503051758 | 37804394.0
0.6215295791625977 | 37359059.0
0.6190891265869141 | 29648361.0


### Listing embeddings

In [9]:
listings = pd.read_csv(raw+'listings.csv.gz', sep=',')
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5402 entries, 0 to 5401
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5402 non-null   int64  
 1   listing_url                                   5402 non-null   object 
 2   scrape_id                                     5402 non-null   int64  
 3   last_scraped                                  5402 non-null   object 
 4   name                                          5402 non-null   object 
 5   description                                   5392 non-null   object 
 6   neighborhood_overview                         3908 non-null   object 
 7   picture_url                                   5402 non-null   object 
 8   host_id                                       5402 non-null   int64  
 9   host_url                                      5402 non-null   o

In [10]:
# choose columns to generate embeddings.
listings = listings[['id','listing_url','name','description','neighborhood_overview','picture_url', 
'property_type','room_type','accommodates','bathrooms','bathrooms_text',                               
'bedrooms','beds','amenities','price','minimum_nights','maximum_nights','review_scores_rating',                         
'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
'review_scores_communication','review_scores_location']]

listings.fillna('0', inplace=True)

listings.reset_index(drop = True, inplace = True)

In [11]:
import string
def remove_punc(sample_str):
    # Create translation table in which special charcters
    # are mapped to empty string
    translation_table = str.maketrans('', '', string.punctuation)
    # Remove special characters from the string using translation table
    sample_str = sample_str.translate(translation_table)
    return sample_str

In [12]:
listings['words_features'] = listings['amenities'].apply(remove_punc)

for ind in listings.index:
     listings['review_scores_rating'][ind] = (float(listings['review_scores_rating'][ind]) + float(listings['review_scores_accuracy'][ind]) + float(listings['review_scores_cleanliness'][ind]) + float(listings['review_scores_checkin'][ind]) + float(listings['review_scores_communication'][ind]) + float(listings['review_scores_location'][ind]))
     listings['review_scores_rating'][ind] = (listings['review_scores_rating'][ind])/6
     listings['words_features'][ind] = 'amenities:'+listings['words_features'][ind] +'description:'+  listings['description'][ind] +'neighborhood_overview:'+  listings['neighborhood_overview'][ind]+'property_type:'+  listings['property_type'][ind]+'room_type:'+  listings['room_type'][ind]+'accommodates:'+  str(listings['accommodates'][ind])+'bedrooms:'+  str(listings['bedrooms'][ind])+'beds:'+  str(listings['beds'][ind])+'price range:'+  listings['price'][ind]
listings = listings.rename(columns={"review_scores_rating": "overall_rating"})
listings["overall_rating"].fillna(0, inplace=True)

<ipython-input-12-8769852d1023>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['review_scores_rating'][ind] = (float(listings['review_scores_rating'][ind]) + float(listings['review_scores_accuracy'][ind]) + float(listings['review_scores_cleanliness'][ind]) + float(listings['review_scores_checkin'][ind]) + float(listings['review_scores_communication'][ind]) + float(listings['review_scores_location'][ind]))
<ipython-input-12-8769852d1023>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings['review_scores_rating'][ind] = (listings['review_scores_rating'][ind])/6
<ipython-input-12-8769852d1023>:6: SettingW

In [13]:
text_corpus_model = create_embeddings(listings['words_features'].values,'list')

new_doc = gensim.parsing.preprocessing.preprocess_string("private room dishwasher")
test_doc_vector = text_corpus_model.infer_vector(new_doc)
sims = text_corpus_model.docvecs.most_similar(positive = [test_doc_vector])
for s in sims:
    print(f"{(s[1])} | {listings['listing_url'].iloc[s[0]]}")

0.4589652717113495 | https://www.airbnb.com/rooms/25991654
0.4400947391986847 | https://www.airbnb.com/rooms/40247038
0.4249565899372101 | https://www.airbnb.com/rooms/8641456
0.4234083890914917 | https://www.airbnb.com/rooms/51961840
0.41909921169281006 | https://www.airbnb.com/rooms/49927889
0.4160788655281067 | https://www.airbnb.com/rooms/16685383
0.41239070892333984 | https://www.airbnb.com/rooms/32729549
0.4093344509601593 | https://www.airbnb.com/rooms/39642322
0.4028607904911041 | https://www.airbnb.com/rooms/28220320
0.40280336141586304 | https://www.airbnb.com/rooms/21593642


<ipython-input-13-75f26069de46>:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = text_corpus_model.docvecs.most_similar(positive = [test_doc_vector])


### Help link Embeddings

In [14]:
import json
from typing import Any, Text, Dict, List
import torch
from bert_serving.client import BertClient
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
import numpy as np
from sentence_transformers import SentenceTransformer

# sentence embedding selection
sentence_transformer_select=True
pretrained_model='stsb-roberta-large' # Refer: https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md
score_threshold = 0.70  # This confidence scores can be adjusted based on your need!!

In [15]:

def encode_standard_question(sentence_transformer_select=True, pretrained_model='bert-base-nli-mean-tokens'):
    """
    This will encode all the questions available in question database into sentence embedding. The result will be stored into numpy array for comparision purpose.
    """
    if sentence_transformer_select:
        bc = SentenceTransformer(pretrained_model)
    else:
        bc = BertClient(check_version=False)
    data = json.load(open(raw+"/faq.json", "rt", encoding="utf-8"))
    standard_questions = [each['q'].replace('-',' ') for each in data]
    print("Standard question size", len(standard_questions))
    print("Start to calculate encoder....")
    if sentence_transformer_select:
        standard_questions_encoder = torch.tensor(bc.encode(standard_questions)).numpy()
    else:
        standard_questions_encoder = bc.encode(standard_questions)
    np.save(root+"questions_embedding", standard_questions_encoder)
    standard_questions_encoder_len = np.sqrt(np.sum(standard_questions_encoder * standard_questions_encoder, axis=1))
    np.save(root+"questions_embedding_len", standard_questions_encoder_len)


encode_standard_question(sentence_transformer_select,pretrained_model)

Standard question size 86
Start to calculate encoder....
